In [1]:
from poloniex import Poloniex

polo = Poloniex()

In [2]:
markets = polo.returnTicker()
pairs = []
for name, thinks in markets.items():
    pairs.append(name)
print pairs

[u'USDT_REP', u'BTC_CVC', u'BTC_XVC', u'BTC_PINK', u'BTC_SYS', u'BTC_EMC2', u'BTC_RADS', u'BTC_SC', u'BTC_MAID', u'BTC_BCH', u'BTC_GNT', u'BTC_BCN', u'BTC_REP', u'BTC_BCY', u'BTC_GNO', u'XMR_NXT', u'USDT_ZEC', u'BTC_FCT', u'BTC_GAS', u'USDT_ETH', u'USDT_BTC', u'BTC_LBC', u'BTC_DCR', u'USDT_ETC', u'ETH_OMG', u'BTC_AMP', u'BTC_XPM', u'BTC_NXT', u'BTC_VTC', u'ETH_STEEM', u'XMR_BLK', u'BTC_PASC', u'XMR_ZEC', u'BTC_GRC', u'BTC_NXC', u'BTC_BTCD', u'BTC_LTC', u'BTC_DASH', u'BTC_STORJ', u'ETH_ZEC', u'BTC_ZEC', u'BTC_BURST', u'ETH_ZRX', u'BTC_BELA', u'BTC_STEEM', u'BTC_ETC', u'BTC_ETH', u'BTC_HUC', u'BTC_STRAT', u'BTC_LSK', u'BTC_EXP', u'BTC_CLAM', u'ETH_REP', u'XMR_DASH', u'ETH_CVC', u'USDT_BCH', u'BTC_ZRX', u'USDT_DASH', u'BTC_BLK', u'BTC_XRP', u'USDT_NXT', u'BTC_NEOS', u'BTC_OMG', u'BTC_BTS', u'BTC_DOGE', u'ETH_GNT', u'BTC_SBD', u'ETH_GNO', u'BTC_XCP', u'USDT_LTC', u'BTC_BTM', u'USDT_XMR', u'ETH_LSK', u'BTC_OMNI', u'BTC_NAV', u'BTC_FLDC', u'ETH_BCH', u'BTC_XBC', u'BTC_DGB', u'XMR_BTCD', u'BT

In [3]:
# funciones necesarias para el manejo del tiempo
from datetime import datetime, timedelta
from time import sleep
from tusp import string2ts
from bot import prepareData
import numpy as np
import pylab as pl
%matplotlib inline

class PairData:
    
    def __init__(self, pair, period, weeks = 4.3*3):
        
        self.pair = pair
        self.period = period
        
        tf = datetime.now()
        self.tf = tf
        end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
        self.end = end
        delta = timedelta(weeks = weeks)
        self.delta = delta
        to = tf-delta
        self.to = to
        start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
        self.start = start
        
        df = prepareData(pair=pair, start=start, end=end, period=int(period))
        
        self.df = df
        self.market_return()
    
    def market_return(self):
        """Calcula el retorno del mercado vía log_retornos
        """
        
        serie = self.df["close"]

        # calculando los log retornos
        log_return = np.log(serie).diff().fillna(0)

        # calculando log retorno acumulado
        cum_logr = log_return.cumsum()
        cum_r = np.exp(cum_logr) - 1
        
        self.l_return = log_return
        self.m_return = cum_r
        
    
    def sortino(self, log = False):
        """ Calcula en ratio de Sortino
        
        """
        
        if log:
            rets = self.l_return
         
        else:
            rets = self.m_return

        semi_var = rets[rets < 0] ** 2
        semi_var = semi_var.sum()/len(rets)
        sortino = np.sqrt(semi_var)

        return rets.mean()/sortino


dgb = PairData("BTC_BCH", 3600*4)

print dgb.sortino(log = False)

1.21846790972


In [13]:
sortinos = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4)
    sortino = asset.sortino(log = False)
    sortinos.append([pair, sortino])
    #print sortino

sortinos.sort(key = lambda l:l[1], reverse=True)

print sortinos[0:6]

[[u'BTC_STR', 934.47267380848712], [u'BTC_VTC', 97.72554004753276], [u'BTC_NXT', 3.0918694011520347], [u'BTC_EMC2', 1.9476432651930382], [u'BTC_BCH', 1.2616813132484126], [u'BTC_SBD', 1.0184861132270357]]


In [16]:
sortinos = [x for x in sortinos if x[0].split("_")[0] == "BTC"]
print sortinos[0:7]

[[u'BTC_STR', 934.47267380848712], [u'BTC_VTC', 97.72554004753276], [u'BTC_NXT', 3.0918694011520347], [u'BTC_EMC2', 1.9476432651930382], [u'BTC_BCH', 1.2616813132484126], [u'BTC_SBD', 1.0184861132270357], [u'BTC_POT', 0.52065066531293269]]


In [8]:
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]

In [9]:
from indicators import rsi
rsi_list = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*24, weeks = 4.3*6)
    RSI1, RSI2 = rsi(asset.df["close"])
    rsi_list.append([pair, RSI1[-1]])
    #print sortino

rsi_list.sort(key = lambda l:l[1], reverse=False)

print rsi_list[0:6]

indicators.py:29: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=True,com=14).mean()
  roll_up1 = pd.stats.moments.ewma(up, window_length)
indicators.py:30: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=True,com=14).mean()
  roll_down1 = pd.stats.moments.ewma(down.abs(), window_length)
indicators.py:42: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,center=False).mean()
  roll_up2 = pd.rolling_mean(up, window_length)
indicators.py:43: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,center=False).mean()
  roll_down2 = pd.rolling_mean(down.abs(), window_length)


[[u'BTC_EMC2', 44.878882020049012], [u'BTC_NXT', 45.184407918716602], [u'BTC_VTC', 46.667719222243846], [u'BTC_ZEC', 53.622935319825956], [u'BTC_BCH', 54.418331556867471], [u'BTC_ETC', 54.837203747099231]]


[[u'BTC_STR', 87.587091357153909], [u'BTC_XRP', 83.102721235508071], [u'BTC_ZRX', 79.847489227947847], [u'BTC_AMP', 77.322103319399091], [u'BTC_PASC', 76.876178439913133], [u'BTC_VIA', 75.944846852540195]]
